
# Notebook CoT aplicada

Clase aplicada de Zero-shot CoT + Few-shot CoT con feedback loop para un agente conversacional coqueto y respetuoso.


In [1]:

from __future__ import annotations

import json
import os
import sys
from pathlib import Path

from dotenv import load_dotenv
from openai import OpenAI

ROOT = Path.cwd()
sys.path.append(str((ROOT / "02-prompting").resolve()))
from common.rubrica import evaluar_salida

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("Falta OPENAI_API_KEY en .env")

model = os.getenv("OPENAI_MODEL", "gpt-4o-mini")
client = OpenAI(api_key=api_key)
print(f"Modelo activo: {model}")


Modelo activo: gpt-4o-mini


In [2]:

perfil = {
    "tipo_persona": "product manager de salud digital",
    "gustos": ["running", "cafes tranquilos", "podcasts de tecnologia"],
    "estilo": "directo y curioso",
    "contexto": "primera conversacion despues de un match",
}

system_prompt = (
    "Eres coach conversacional. Tono coqueto y respetuoso. "
    "Prohibido lenguaje explicito, manipulador o de presion. "
    "No uses emojis."
)

user_prompt = f"""
Perfil:
{json.dumps(perfil, ensure_ascii=False, indent=2)}

Haz razonamiento en 4 pasos y luego entrega JSON exacto:
{{
  "chain_of_thought": ["...", "...", "...", "..."],
  "opener": "...",
  "follow_up": "...",
  "tone_notes": ["...", "..."],
  "avoid": ["...", "..."]
}}
"""

resp = client.chat.completions.create(
    model=model,
    temperature=0.6,
    response_format={"type": "json_object"},
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ],
)

base = json.loads(resp.choices[0].message.content)
print(json.dumps(base, ensure_ascii=False, indent=2))


{
  "chain_of_thought": [
    "El perfil indica que la persona es product manager en salud digital, lo que sugiere que tiene un enfoque en la tecnología y la innovación en ese campo.",
    "Sus gustos en running, cafés tranquilos y podcasts de tecnología ofrecen oportunidades para iniciar una conversación que refleje intereses compartidos.",
    "El estilo directo y curioso sugiere que apreciará preguntas que sean claras y que fomenten un intercambio genuino.",
    "Es importante mantener un tono coqueto y respetuoso, asegurando que la conversación fluya de manera natural y amena."
  ],
  "opener": "Hola, parece que tenemos un match interesante. ¿Qué te llevó a convertirte en product manager de salud digital?",
  "follow_up": "Y ya que mencionas que te gusta el running, ¿cuál es tu ruta favorita? Siempre estoy buscando nuevas recomendaciones.",
  "tone_notes": [
    "coqueto",
    "respetuoso"
  ],
  "avoid": [
    "lenguaje explícito",
    "manipulación o presión"
  ]
}


In [3]:

evaluacion_base = evaluar_salida(perfil, base)
print(json.dumps(evaluacion_base, ensure_ascii=False, indent=2))


{
  "scores": {
    "personalizacion": 5,
    "naturalidad": 8,
    "respeto": 9,
    "accionable": 8
  },
  "promedio": 7.5,
  "feedback": [
    "Incluye referencias más claras a los gustos del perfil."
  ]
}


In [4]:

prompt_mejora = f"""
Perfil:
{json.dumps(perfil, ensure_ascii=False, indent=2)}

Salida actual:
{json.dumps(base, ensure_ascii=False, indent=2)}

Rubrica:
{json.dumps(evaluacion_base, ensure_ascii=False, indent=2)}

Mejora la respuesta para subir el promedio de la rubrica.
No uses emojis.
Devuelve JSON con la misma estructura.
"""

resp2 = client.chat.completions.create(
    model=model,
    temperature=0.3,
    response_format={"type": "json_object"},
    messages=[
        {"role": "system", "content": "Eres un revisor de calidad de prompting."},
        {"role": "user", "content": prompt_mejora},
    ],
)

mejorada = json.loads(resp2.choices[0].message.content)
evaluacion_mejorada = evaluar_salida(perfil, mejorada)

print("Version mejorada")
print(json.dumps(mejorada, ensure_ascii=False, indent=2))
print("Evaluacion mejorada")
print(json.dumps(evaluacion_mejorada, ensure_ascii=False, indent=2))


Version mejorada
{
  "chain_of_thought": [
    "El perfil indica que la persona es product manager en salud digital, lo que sugiere que tiene un enfoque en la tecnología y la innovación en ese campo.",
    "Sus gustos en running, cafés tranquilos y podcasts de tecnología ofrecen múltiples oportunidades para iniciar una conversación que refleje intereses compartidos.",
    "El estilo directo y curioso sugiere que apreciará preguntas que sean claras y que fomenten un intercambio genuino.",
    "Es importante mantener un tono coqueto y respetuoso, asegurando que la conversación fluya de manera natural y amena.",
    "Incorporar preguntas sobre sus gustos específicos puede hacer que la conversación sea más atractiva y personalizada."
  ],
  "opener": "Hola, parece que tenemos un match interesante. ¿Qué te inspiró a convertirte en product manager de salud digital? Me imagino que es un campo fascinante.",
  "follow_up": "Y ya que mencionas que te gusta el running, ¿tienes alguna ruta favorit